In [0]:
# Daily Sales Genie view

In [0]:
%sql
create or replace view main.retail_gold.vw_sales_daily as
select
    d.date                as order_date,
    d.date_sk             as order_date_sk,
    d.year                as order_year,
    d.quarter             as order_quarter,
    d.month               as order_month,
    d.month_name          as order_month_name,
    d.week                as order_week,
    d.day_of_week         as order_dow,
    d.dow_name            as order_dow_name,

    f.order_line_id,
    f.order_id,
    f.product_id,
    f.customer_id,
    f.geo_id,

    p.product_name,
    p.subcategory,
    p.category,

    g.region,
    g.state,
    g.city,
    g.postal_code,

    c.customer_name,
    c.segment as customer_segment,

    f.sales,
    f.profit,
    f.quantity,
    f.discount,

    case when f.sales <> 0 then f.profit / f.sales end as profit_margin

from main.retail_gold.FactSales   f
join main.retail_gold.DateDim     d on f.order_date_sk = d.date_sk
join main.retail_gold.ProductDim  p on f.product_id    = p.product_id
join main.retail_gold.GeoDim      g on f.geo_id        = g.geo_id
left join main.retail_gold.CustomerDim c on f.customer_id = c.customer_id;

In [0]:
# Monthly sales Genie view

In [0]:
%sql
create or replace view main.retail_gold.vw_sales_monthly as
select
    d.year,
    d.quarter,
    d.month,
    d.month_name,
    sum(f.sales)                  as sales,
    sum(f.profit)                 as profit,
    sum(f.quantity)               as quantity,
    sum(f.discount)               as discount,
    count(distinct f.order_id)    as orders,
    case when sum(f.sales) <> 0
         then sum(f.profit) / sum(f.sales) end        as profit_margin,
    case when count(distinct f.order_id) <> 0
         then sum(f.sales) / count(distinct f.order_id) end as avg_order_value
from main.retail_gold.FactSales f
join main.retail_gold.DateDim   d on f.order_date_sk = d.date_sk
group by d.year, d.quarter, d.month, d.month_name;


In [0]:
# Topline metrics Genie view

In [0]:
%sql
create or replace view main.retail_gold.vw_topline as
with agg as (
  select
      d.year,
      d.quarter,
      d.month,
      d.month_name,
      sum(f.sales)                as sales,
      sum(f.profit)               as profit,
      sum(f.quantity)             as quantity,
      sum(f.discount)             as discount,
      count(distinct f.order_id)  as orders
  from main.retail_gold.FactSales f
  join main.retail_gold.DateDim   d on f.order_date_sk = d.date_sk
  group by grouping sets (
    (d.year),
    (d.year, d.quarter),
    (d.year, d.month, d.month_name)
  )
)
select
    case
      when year is not null and quarter is null and month is null then 'year'
      when year is not null and quarter is not null and month is null then 'quarter'
      when year is not null and month is not null then 'month'
      else 'unknown'
    end as period_type,
    year, quarter, month, month_name,
    sales, profit, quantity, discount, orders,
    case when sales  <> 0 then profit / sales end   as profit_margin,
    case when orders <> 0 then sales  / orders end  as avg_order_value
from agg;